# 1. instrall libraries

In [2]:
!pip install cython

     ------------------------------------- 985.1/985.1 KB 12.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\bae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install git+https://github.com/mayukh18/reco

  Cloning https://github.com/mayukh18/reco to c:\users\bae\appdata\local\temp\pip-req-build-96tu95cz
  Resolved https://github.com/mayukh18/reco to commit 7f0bfc40edc6c21afef4314ee30605766281990b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for reco: filename=reco-0.2.1-cp310-cp310-win_amd64.whl size=1590067 sha256=bce943bc85f504d4b50ef5af314765ca2edf71e056add440e248572ad2eae831
  Stored in directory: C:\Users\bae\AppData\Local\Temp\pip-ephem-wheel-cache-kqmolc57\wheels\08\84\1f\4f54fb9df6f7483c6d24d46bca75446401623e035556af4c70
Successfully built reco


  Running command git clone --filter=blob:none --quiet https://github.com/mayukh18/reco 'C:\Users\bae\AppData\Local\Temp\pip-req-build-96tu95cz'
You should consider upgrading via the 'C:\Users\bae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install reco

You should consider upgrading via the 'C:\Users\bae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
import glob
import reco
from tqdm import tqdm
import datetime
from collections import Counter

# 2. transaction dataset 불러오기

In [7]:
data = pd.read_csv("./transactions_train.csv", dtype={'article_id':str})
data["t_dat"] = pd.to_datetime(data["t_dat"])
data.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


### 2.1 transaction dataset을 날짜로 분류

In [8]:
train1 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,16)) & (data['t_dat'] < datetime.datetime(2020,9,23))]
train2 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,8)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train3 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,31)) & (data['t_dat'] < datetime.datetime(2020,9,8))]
train4 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,23)) & (data['t_dat'] < datetime.datetime(2020,8,31))]

positive_items_per_user1 = train1.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user2 = train2.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user3 = train3.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user4 = train4.groupby(['customer_id'])['article_id'].apply(list)

train = pd.concat([train1, train2], axis=0)
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

_, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])

user_group = pd.concat([train1, train2, train3, train4], axis=0).groupby(['customer_id'])['article_id'].apply(list)

### 2.2 'feedback' column을 추가하여 각 고객과 제품별 인기도를 구한다.

In [9]:
# SVD
train = pd.concat([train1, train2, train3, train4], axis=0)
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days**2)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

train['feedback'] = 1
train = train.groupby(['customer_id', 'article_id']).sum().reset_index()

train['feedback'] = train.apply(lambda row: row['feedback']/popular_items_group[row['article_id']], axis=1)

train['feedback'] = train['feedback'].apply(lambda x: 5.0 if x>5.0 else x)
train.drop(['price', 'sales_channel_id'], axis=1, inplace=True)
train['feedback'].describe()

count    1.020512e+06
mean     7.597183e-01
std      1.431037e+00
min      6.119724e-03
25%      4.992918e-02
50%      1.387718e-01
75%      5.250353e-01
max      5.000000e+00
Name: feedback, dtype: float64

In [10]:
train = train.sample(frac=1).reset_index(drop=True)
train.head()

,customer_id,article_id,pop_factor,feedback
0,e69e88c32307b9b25f5a3d9652d5ff0f100698ce3ebfcb...,0903735001,0.001479,0.691657
1,383cae9bf5af0681eb0c5d016f2ef2e5b50c112fee7aa1...,0815550001,0.001736,5.000000
2,764fe28ed006f8413d8d8ccad3c24c2c0a2df51ae949a4...,0769102040,0.002770,2.179690
3,3d9cf1d1c74ed4435d064b4c71e45305e19901a2c59c45...,0610776105,0.001189,0.057682
4,f7c660bcb16f62b79329582c969eb155e47d70f4ba776a...,0662888007,0.005102,1.898106


# 3.FunkSVD를 이용한 DATA 학습

In [15]:
from reco.recommender import FunkSVD

f = FunkSVD(k=64, learning_rate=0.002, regularizer = 0.05, iterations = 300, method = 'stochastic', bias=True)
f.fit(X=train, formatizer={'user':'customer_id', 'item':'article_id', 'value':'feedback'},verbose=True)

Epoch 0: Error: 0.8085307182535995
Epoch 1: Error: 0.6533128865977235
Epoch 2: Error: 0.5799840168153515
Epoch 3: Error: 0.5345877840539213
Epoch 4: Error: 0.5023188398218753
Epoch 5: Error: 0.4773419405148281
Epoch 6: Error: 0.4568940832471957
Epoch 7: Error: 0.4395206417191778
Epoch 8: Error: 0.4244019060307467
Epoch 9: Error: 0.4110306058563546
Epoch 10: Error: 0.39906836981658683
Epoch 11: Error: 0.3882724001305593
Epoch 12: Error: 0.3784636102127844
Epoch 13: Error: 0.3695161608487101
Epoch 14: Error: 0.36132512897480507
Epoch 15: Error: 0.3537978635318129
Epoch 16: Error: 0.34686203071249316
Epoch 17: Error: 0.340450261956018
Epoch 18: Error: 0.33450473857963436
Epoch 19: Error: 0.3289745887950932
Epoch 20: Error: 0.32381625309414297
Epoch 21: Error: 0.31898700648101164
Epoch 22: Error: 0.31445217284982097
Epoch 23: Error: 0.310180493360066
Epoch 24: Error: 0.3061453085800974
Epoch 25: Error: 0.30232543769731857
Epoch 26: Error: 0.2987006282091108
Epoch 27: Error: 0.2952517424554

# 4. Submission

In [16]:
submission = pd.read_csv("./sample_submission.csv")
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [19]:
def get_most_freq_next_item(user_group):
    next_items = {}
    for user in tqdm(user_group.keys()):
        items = user_group[user]
        for i,item in enumerate(items[:-1]):
            if item not in next_items:
                next_items[item] = []
            if item != items[i+1]:
                next_items[item].append(items[i+1])

    pred_next = {}
    for item in next_items:
        if len(next_items[item]) >= 5:
            most_common = Counter(next_items[item]).most_common()
            ratio = most_common[0][1]/len(next_items[item])
            if ratio >= 0.1:
                pred_next[item] = most_common[0][0]

    return pred_next

pred_next = get_most_freq_next_item(user_group)

100%|██████████| 250619/250619 [00:01<00:00, 199699.47it/s]


In [20]:
outputs = []
cnt = 0

popular_items = list(popular_items)
userindexes = {f.users[i]:i for i in range(len(f.users))}

for user in tqdm(submission['customer_id']):
    user_output = []
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}

        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]

    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}

        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]

    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}

        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]

    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}

        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]

    user_output += [pred_next[item] for item in user_output if item in pred_next and pred_next[item] not in user_output]

    user_output += list(popular_items[:12 - len(user_output)])
    outputs.append(user_output)

str_outputs = []
for output in outputs:
    str_outputs.append(" ".join([str(x) for x in output]))

100%|██████████| 1371980/1371980 [06:12<00:00, 3684.89it/s]


In [21]:
submission['prediction'] = str_outputs
submission.to_csv("submissions.csv", index=False)

In [22]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0924243002 0918522001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0918522001 0751471001 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0805000001 0924243001 0924243002 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0918522001 0751471001 04...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0918522001 0751471001 04...
